## Retrieve NWM retrospective mean spatial precipitation (MAP) from version 2.1

* This code retrieves precipitation data from the NWM retrospective dataset stored in AWS (https://registry.opendata.aws/nwm-archive/) and computes the average over a basin area. 
* Basin area is defined by a shapefile poligon.
* This code was adapted from Garousi-Nejad, I., A. M. Castronova (2024). Analysis of precipitation data across the Logan River Watershed for the Year 2010, HydroShare, http://www.hydroshare.org/resource/b1379f00121e456f958f9e22e913aa8a

In [ ]:
import os
import pandas
import xarray
import s3fs
import hvplot.xarray
import geopandas
import numpy
import pyproj
import rioxarray 
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from datetime import datetime
import contextily as cx

from dask.distributed import Client
client = Client()
client

In [ ]:
# Path where the recipitation data lives
s3_path = 's3://noaa-nwm-retrospective-2-1-zarr-pds/precip.zarr'

In [ ]:
# Connect to S3
s3 = s3fs.S3FileSystem(anon=True)
store = s3fs.S3Map(root=s3_path, s3=s3, check=False)

In [ ]:
%%time
# load the dataset
ds = xarray.open_zarr(store=store, consolidated=True)

#### Defining projections

In [ ]:
# create a 2D grid of coordinate values
X, Y = numpy.meshgrid(ds.x.values, ds.y.values)

# define the input crs
wrf_proj = pyproj.Proj(proj='lcc',
                       lat_1=30.,
                       lat_2=60., 
                       lat_0=40.0000076293945, lon_0=-97., # Center point
                       a=6370000, b=6370000) 

# Define the output crs
wgs_proj = pyproj.Proj(proj='latlong', datum='WGS84')

# transform X, Y into Lat, Lon
transformer = pyproj.Transformer.from_crs(wrf_proj.crs, wgs_proj.crs)
lon, lat = transformer.transform(X, Y)

# add geographical coordinate values (log and lat) to the dataset
ds = ds.assign_coords(lon = (['y', 'x'], lon))
ds = ds.assign_coords(lat = (['y', 'x'], lat))

#add crs to file
ds.rio.write_crs(ds.crs.attrs['spatial_ref'], inplace=True
                ).rio.set_spatial_dims(x_dim="x",
                                       y_dim="y",
                                       inplace=True,
                                       ).rio.write_coordinate_system(inplace=True)

# make sure the data is sorted by time
ds = ds.sortby('time')

In [ ]:
# define a target coordinate system to convert the geometry data into the projection of our forcing data
target_crs = wrf_proj

#### Test of MAP computation with one basin [optional]

In [ ]:
timerange = slice('2011-04-28 17:00:00','2011-04-28 17:00:00')
max_lon = ds["x"].max()
max_lat = ds["y"].max()
dat = ds.sel(time=timerange,x=slice(1e6,max_lon),y=slice(0,max_lat)).RAINRATE.persist() # Only the NE US
#dat = ds.sel(time=timerange).RAINRATE.persist() # Full CONUS

In [ ]:
dat.plot()

In [ ]:
# Load basin shapefile
shp_path = './Input/Shp/'
fname = 'ST_W5.shp'
shapefile = shp_path+fname

shp = geopandas.read_file(shapefile)
shp_crs = shp.crs

In [ ]:
cx.providers.CartoDB.keys()

In [ ]:
ax = shp.plot(figsize=(9, 9), alpha=0.5, edgecolor='k')
cx.add_basemap(ax,crs=shp.crs,source=cx.providers.CartoDB.Voyager)

In [ ]:
# Reproject basins to AORC projection
shp_prj = shp.to_crs(target_crs.crs)

In [ ]:
# With this method all the pixels that are touched in any % by the limit are included
clipped = dat.rio.clip(shp_prj.geometry, all_touched=True, drop= True)

In [ ]:
# With this method if the centroid is not within the limit, the cell is not included
clipped = dat.rio.clip(shp_prj.geometry.values,
                 shp_prj.crs,
                 drop=True,
                 invert=False) 

In [ ]:
# Compute spatial mean
map = clipped.groupby("time").mean(["y", "x"])
map.to_dataframe()

In [ ]:
# Plot to double check the result
f, ax = plt.subplots(1,figsize=(9, 9))
clipped.plot(ax=ax)
shp_prj.plot(ax=ax,cmap=None,facecolor="none", edgecolor='k')
plt.show()

#### Get MAP values for all basins over the full retrospective period by time chunks

In [ ]:
# slice all data in time chunks
start_date = datetime.strptime("1979-02-01 00:00:00", "%Y-%m-%d %H:%M:%S")
end_date = datetime.strptime("2021-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")

date_list = pandas.date_range(start_date, end_date, periods=11) # Adjust number of periods as needed
date_list

In [ ]:
# Setting paths and names
shp_path = './Input/Shp/'
savePath = './Output/'
bsns_name = ["ST_RB","ST_W3","ST_W5","ST_WB"]

In [ ]:
# Loop to process the data in time chunks
max_lon = ds["x"].max()
max_lat = ds["y"].max()

for i in range(len(date_list)-1):
    print("Processing block of dates from",date_list[i], "to",date_list[i+1])
    timerange = slice(str(date_list[i]), str(date_list[i+1]))
    print(timerange)
    dat = ds.sel(time=timerange,x=slice(1e6,max_lon),y=slice(0,max_lat)).RAINRATE.persist()
    
    # Loop through the sites
    for name in bsns_name:
        shapefile = (shp_path+name+".shp")
        print(shapefile)
        shp = geopandas.read_file(shapefile)
        
        # Reproject shapefile to AORC projection
        shp_prj = shp.to_crs(target_crs.crs)
        
        # Clip or extract the data within the basin
        clipped = dat.rio.clip(shp_prj.geometry.values,shp_prj.crs,drop=True,invert=False)
        
        # Calculate spatial mean per basin
        mean_sprecip = clipped.groupby("time").mean(["y", "x"]).to_dataframe()
        
        # Add a column the basin identifier
        mean_sprecip['Basin'] = name
        
        # Save as csv
        mean_sprecip.to_csv(f'{savePath}NWM_AORC_MAP_{name}_{str(i)}.csv')
        print("Saving data...Done")
        
        # Delete unnecesary data to save memory
    del(clipped,dat,mean_sprecip) 